# Vision models - model training using transfer learning



## Python imports

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import sklearn.model_selection
import tensorflow as tf
import tensorflow_hub as hub
import time

2025-05-16 17:06:02.528770: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-16 17:06:02.572428: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-16 17:06:02.573281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-16 17:06:03.225668: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Configuration

In [2]:
import ipywidgets as widgets

# MODEL_CHARACTERISTICS
MODELS = {
    "InceptionV1" : {
        "url": "https://www.kaggle.com/models/google/inception-v1/TensorFlow2/feature-vector/2",
        "dim": 224
    },
    "InceptionV2" : {
        "url": "https://www.kaggle.com/models/google/inception-v2/TensorFlow2/feature-vector/2",
        "dim": 224
    },
    "MobileNetV1" : {
        "url": "https://www.kaggle.com/models/google/mobilenet-v1/TensorFlow2/100-224-feature-vector/2",
        "dim": 224
    },
    "MobileNetV2" : {
        "url": "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/100-224-feature-vector/2",
        "dim": 224
    },
    "MobileNetV3" : {
        "url": "https://www.kaggle.com/models/google/mobilenet-v3/TensorFlow2/small-100-224-feature-vector/1",
        "dim": 224
    },
    "NasNetMobile" : {
        "url": "https://www.kaggle.com/models/google/nasnet/TensorFlow2/mobile-feature-vector/2",
        "dim": 224
    },
    "ResNetV1" : {
        "url": "https://www.kaggle.com/models/google/resnet-v1/TensorFlow2/50-feature-vector/2",
        "dim": 224
    },
    "ResNetV2" : {
        "url": "https://www.kaggle.com/models/google/resnet-v2/TensorFlow2/50-feature-vector/2",
        "dim": 224
    },
}
P_DIM=224
NUM_CLASSES=21
CT_CFG = widgets.FloatSlider(description="Convergenge threshold:", value=1.0, min=0.25, max=1.00, step=0.01)
DROPOUT_CFG = widgets.FloatSlider(description="Dropout (0: no dropout):", value=0.2, min=0.0, max=0.5, step=0.1)
MAX_STEPS_CFG = widgets.IntSlider(description="Max. training steps:", value=25, min=1, max=100, step=10)
BATCH_SIZE_CFG = widgets.IntSlider(description="Batch size:", value=32, min=32, max=500)
display(widgets.VBox([CT_CFG, DROPOUT_CFG, MAX_STEPS_CFG, BATCH_SIZE_CFG]))

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [3]:

global P_CT
P_CT = CT_CFG.value
P_DROPOUT = DROPOUT_CFG.value
P_MAX_STEPS = MAX_STEPS_CFG.value
P_BATCH_SIZE = BATCH_SIZE_CFG.value
print('','Input dim:', P_DIM, '\n',
      'Num classes', NUM_CLASSES, '\n',
      'Conv. thr.:', P_CT, '\n',
      'Dropout:', P_DROPOUT, '\n',
      'Max steps:', P_MAX_STEPS, '\n',
      'Batch size:', P_BATCH_SIZE)

 Input dim: 224 
 Num classes 21 
 Conv. thr.: 1.0 
 Dropout: 0.2 
 Max steps: 25 
 Batch size: 32


## Read train data set

In [4]:
train_df = pd.read_csv('datasets/vision/train.tsv', sep='\t')
train_df

,label,set,time,source,image,video
0,AT_CH,train,0,Pixel,video_data/AT_CH/train_Pixel_0000_0000.jpeg,../dados_indoor_location/Pixel/Vision/12182683...
1,AT_CH,train,1,Pixel,video_data/AT_CH/train_Pixel_0001_0031.jpeg,../dados_indoor_location/Pixel/Vision/12182683...
2,AT_CH,train,3,Pixel,video_data/AT_CH/train_Pixel_0003_0091.jpeg,../dados_indoor_location/Pixel/Vision/12182683...
3,AT_CH,train,4,Pixel,video_data/AT_CH/train_Pixel_0004_0121.jpeg,../dados_indoor_location/Pixel/Vision/12182683...
4,AT_CH,train,5,Pixel,video_data/AT_CH/train_Pixel_0005_0151.jpeg,../dados_indoor_location/Pixel/Vision/12182683...
...,...,...,...,...,...,...
3738,AT_I2,train,106,Xiaomi,video_data/AT_I2/train_Xiaomi_0106_3181.jpeg,../dados_indoor_location/Xiaomi/Vision/1653920...
3739,AT_I2,train,107,Xiaomi,video_data/AT_I2/train_Xiaomi_0107_3211.jpeg,../dados_indoor_location/Xiaomi/Vision/1653920...
3740,AT_I2,train,108,Xiaomi,video_data/AT_I2/train_Xiaomi_0108_3241.jpeg,../dados_indoor_location/Xiaomi/Vision/1653920...
3741,AT_I2,train,109,Xiaomi,video_data/AT_I2/train_Xiaomi_0109_3271.jpeg,../dados_indoor_location/Xiaomi/Vision/1653920...


## Define image data generator for train dataset 

In [5]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

train_dataset = datagen.flow_from_dataframe(
    train_df,
    target_size=(P_DIM,P_DIM),
    batch_size=P_BATCH_SIZE,
    class_mode='sparse',
    shuffle=True,
    seed=1234567,
    x_col='image',
    y_col='label'
)

Found 3743 validated image filenames belonging to 21 classes.



## Train models

In [7]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global P_CT
        if logs.get('accuracy') >= P_CT:
            self.model.stop_training = True
            
for model_id in MODELS:
    print('==> Deriving model', model_id)
    kl = hub.KerasLayer('models/vision/transfer_learning/' + model_id, trainable=False, input_shape=(P_DIM, P_DIM, 3))
    fl = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')

    if P_DROPOUT > 0.0:
        model = tf.keras.Sequential([ kl, tf.keras.layers.Dropout(P_DROPOUT), fl])
    else:
        model = tf.keras.Sequential([ kl, fl])
    model.summary()
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    t_start = time.time()
    history = model.fit(train_dataset, epochs=P_MAX_STEPS, callbacks = [CustomCallback()])
    t_end = time.time()
    print('Training time', t_end - t_start)
    model.save('models/vision/' + model_id)
    converter = tf.lite.TFLiteConverter.from_saved_model('models/vision/' + model_id) # path to the SavedModel directory
    tflite_model = converter.convert()
    with open('models/vision/' + model_id + '/model.tflite', 'wb') as f:
      f.write(tflite_model)

==> Deriving model InceptionV1
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1024)              5607184   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 21)                21525     
                                                                 
Total params: 5628709 (21.47 MB)
Trainable params: 21525 (84.08 KB)
Non-trainable params: 5607184 (21.39 MB)
_________________________________________________________________
Epoch 1/25
117/117 [==============================] - 42s 339ms/step - loss: 1.9153 - accuracy: 0.4956
Epoch 2/25
117/117 [==============================] - 40s 341ms/step - loss: 1.0971 - accuracy: 0.7232
Epoch 3/25
117/117 [==================

INFO:tensorflow:Assets written to: models/vision/InceptionV1/assets
2025-05-16 17:26:52.000091: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-05-16 17:26:52.000120: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-05-16 17:26:52.000445: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: models/vision/InceptionV1
2025-05-16 17:26:52.011424: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-05-16 17:26:52.011457: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: models/vision/InceptionV1
2025-05-16 17:26:52.047968: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-05-16 17:26:52.057766: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-05-16 17:26:52.388950: I tensorflow/cc/saved_model/loader.cc

==> Deriving model InceptionV2
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1024)              10173112  
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 21)                21525     
                                                                 
Total params: 10194637 (38.89 MB)
Trainable params: 21525 (84.08 KB)
Non-trainable params: 10173112 (38.81 MB)
_________________________________________________________________
Epoch 1/25
117/117 [==============================] - 53s 434ms/step - loss: 2.0183 - accuracy: 0.4544
Epoch 2/25
117/117 [==============================] - 52s 441ms/step - loss: 1.1318 - accuracy: 0.7128
Epoch 3/25
117/117 [================

INFO:tensorflow:Assets written to: models/vision/InceptionV2/assets
2025-05-16 17:48:52.846725: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-05-16 17:48:52.846756: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-05-16 17:48:52.846976: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: models/vision/InceptionV2
2025-05-16 17:48:52.868310: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-05-16 17:48:52.868345: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: models/vision/InceptionV2
2025-05-16 17:48:52.922601: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-05-16 17:48:53.319174: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: models/vision/InceptionV2
2025-05-16 17:48:53.458041: I tensorflow/cc/s

==> Deriving model MobileNetV1
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1024)              3228864   
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 21)                21525     
                                                                 
Total params: 3250389 (12.40 MB)
Trainable params: 21525 (84.08 KB)
Non-trainable params: 3228864 (12.32 MB)
_________________________________________________________________
Epoch 1/25
117/117 [==============================] - 31s 251ms/step - loss: 1.7237 - accuracy: 0.5170
Epoch 2/25
117/117 [==============================] - 31s 261ms/step - loss: 0.8015 - accuracy: 0.7785
Epoch 3/25
117/117 [==================

...

INFO:tensorflow:Assets written to: models/vision/NasNetMobile/assets
2025-05-16 18:41:20.401616: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-05-16 18:41:20.401643: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-05-16 18:41:20.401870: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: models/vision/NasNetMobile
2025-05-16 18:41:20.479569: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-05-16 18:41:20.479598: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: models/vision/NasNetMobile
2025-05-16 18:41:20.691960: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-05-16 18:41:22.332584: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: models/vision/NasNetMobile
2025-05-16 18:41:22.898235: I tensorflow/

==> Deriving model ResNetV1
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_7 (KerasLayer)  (None, 2048)              23561152  
                                                                 
 dropout_7 (Dropout)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 21)                43029     
                                                                 
Total params: 23604181 (90.04 MB)
Trainable params: 43029 (168.08 KB)
Non-trainable params: 23561152 (89.88 MB)
_________________________________________________________________
Epoch 1/25
117/117 [==============================] - 98s 820ms/step - loss: 1.3307 - accuracy: 0.6196
Epoch 2/25
117/117 [==============================] - 97s 827ms/step - loss: 0.6117 - accuracy: 0.8221
Epoch 3/25
117/117 [==================

INFO:tensorflow:Assets written to: models/vision/ResNetV1/assets
2025-05-16 19:26:18.008035: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-05-16 19:26:18.008073: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-05-16 19:26:18.008279: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: models/vision/ResNetV1
2025-05-16 19:26:18.026678: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-05-16 19:26:18.026709: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: models/vision/ResNetV1
2025-05-16 19:26:18.076354: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-05-16 19:26:18.480486: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: models/vision/ResNetV1
2025-05-16 19:26:18.608821: I tensorflow/cc/saved_model/l

==> Deriving model ResNetV2
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_8 (KerasLayer)  (None, 2048)              23564800  
                                                                 
 dropout_8 (Dropout)         (None, 2048)              0         
                                                                 
 dense_8 (Dense)             (None, 21)                43029     
                                                                 
Total params: 23607829 (90.06 MB)
Trainable params: 43029 (168.08 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________
Epoch 1/25
117/117 [==============================] - 110s 916ms/step - loss: 1.4716 - accuracy: 0.5830
Epoch 2/25
117/117 [==============================] - 108s 922ms/step - loss: 0.6874 - accuracy: 0.8050
Epoch 3/25
117/117 [================

INFO:tensorflow:Assets written to: models/vision/ResNetV2/assets
2025-05-16 20:12:43.564840: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-05-16 20:12:43.564871: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-05-16 20:12:43.565099: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: models/vision/ResNetV2
2025-05-16 20:12:43.579062: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-05-16 20:12:43.579096: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: models/vision/ResNetV2
2025-05-16 20:12:43.632263: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-05-16 20:12:44.117864: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: models/vision/ResNetV2
2025-05-16 20:12:44.273317: I tensorflow/cc/saved_model/l

## Save model and convert it to TFLite format

In [ ]:

label_map = (train_dataset.class_indices)
label_map